In [1]:
import pandas as pd
from datetime import datetime, timedelta
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']


In [ ]:
def search_courses(search_term):
    try:
        # Ensure text index exists
        db.courses.create_index([("title", "text"), ("description", "text")])
    except Exception as e:
        print("Error creating text index:", e)
    
    try:
        results = list(db.courses.find({"$text": {"$search": search_term}}))
        print(f"Found {len(results)} courses matching '{search_term}'")
        return results
    except Exception as e:
        print("Error searching courses:", e)
        return []

# Example usage
matching_courses = search_courses("Python")

In [ ]:
def top_courses_recommendation(limit=5):
    try:
        recommended_courses = db.enrollments.aggregate([
            {"$group": {"_id": "$courseId", "totalEnrollments": {"$sum": 1}}},
            {"$sort": {"totalEnrollments": -1}},
            {"$limit": limit}
        ])
        
        results = []
        for course in recommended_courses:
            try:
                course_info = db.courses.find_one({"courseId": course["_id"]})
                if course_info:
                    results.append({
                        "title": course_info["title"],
                        "totalEnrollments": course["totalEnrollments"]
                    })
            except Exception as e_inner:
                print(f"Error fetching course info for {course['_id']}: {e_inner}")
        
        return results
    
    except Exception as e:
        print("Error generating recommendations:", e)
        return []

# Example usage
top_courses = top_courses_recommendation(limit=5)
for c in top_courses:
    print(c["title"], "-", c["totalEnrollments"], "enrollments")

In [ ]:
from datetime import datetime, timedelta

def archive_old_enrollments(days_old=365):
    """
    Archive enrollments older than a given number of days.

    Args:
        days_old (int): Number of days to determine old enrollments.
    
    Returns:
        int: Number of enrollments archived.
    """
    archive_date = datetime.now() - timedelta(days=days_old)
    
    try:
        old_enrollments = list(db.enrollments.find({"enrolledAt": {"$lt": archive_date}}))
        
        if old_enrollments:
            try:
                db.enrollments_archive.insert_many(old_enrollments)
                db.enrollments.delete_many({"enrolledAt": {"$lt": archive_date}})
                print(f"Archived {len(old_enrollments)} old enrollments")
                return len(old_enrollments)
            except Exception as e_inner:
                print("Error inserting/deleting archived enrollments:", e_inner)
                return 0
        else:
            print("No enrollments to archive")
            return 0
    
    except Exception as e:
        print("Error fetching old enrollments:", e)
        return 0

# Example usage
archived_count = archive_old_enrollments(days_old=365)

In [ ]:
def find_nearby_courses(user_location, max_distance_m=10000):
    # Ensure geospatial index exists
    try:
        db.courses.create_index([("location", "2dsphere")])
    except Exception as e:
        print("Error creating geospatial index:", e)

    # Query for nearby courses
    try:
        nearby_courses = list(db.courses.find({
            "location": {
                "$near": {
                    "$geometry": {"type": "Point", "coordinates": user_location},
                    "$maxDistance": max_distance_m
                }
            }
        }))
        print(f"Found {len(nearby_courses)} nearby courses")
        return nearby_courses
    except Exception as e:
        print("Error fetching nearby courses:", e)
        return []

# Example usage
user_coords = [3.3792, 6.5244]  # longitude, latitude
nearby_courses = find_nearby_courses(user_coords, max_distance_m=10000)